In [ ]:
# Let's import the libraries we will need
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

: 

In [2]:
import os
os.environ["SPARK_HOME"] = "/home/alejandro/Escritorio/venv/lib/python3.12/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"



from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

25/05/12 21:49:29 WARN Utils: Your hostname, Alejandro resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface enp46s0)
25/05/12 21:49:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/12 21:49:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

25/05/12 21:49:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dayofweek, date_format, count, avg, when, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType

# Crear directorio de resultados si no existe
import os
if not os.path.exists("resultados"):
    os.makedirs("resultados")
    print("Directorio 'resultados' creado.")

# Definir el esquema para los datos
schema = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("RatecodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

# Cargar los datos CSV
# Usamos la ruta relativa según la estructura del proyecto descrita
df = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load("data/yellow_tripdata_2016-03.csv")

# Mostrar las primeras filas para verificar que los datos se cargaron correctamente
print("Muestra de los datos originales:")
df.show(5)

# Función auxiliar para determinar el cuadrante basado en coordenadas
def add_quadrants(df):
    # Para Nueva York, aproximadamente:
    # Centro de Manhattan está alrededor de -73.98, 40.75
    # Dividimos en cuadrantes basados en estas coordenadas centrales
    return df.withColumn(
        "pickup_quadrant", 
        when((col("pickup_longitude") == 0) | (col("pickup_latitude") == 0), "Invalid")
        .when((col("pickup_longitude") > -73.98) & (col("pickup_latitude") > 40.75), "NE")
        .when((col("pickup_longitude") > -73.98) & (col("pickup_latitude") <= 40.75), "SE")
        .when((col("pickup_longitude") <= -73.98) & (col("pickup_latitude") > 40.75), "NW")
        .when((col("pickup_longitude") <= -73.98) & (col("pickup_latitude") <= 40.75), "SW")
        .otherwise("Unknown")
    ).withColumn(
        "dropoff_quadrant", 
        when((col("dropoff_longitude") == 0) | (col("dropoff_latitude") == 0), "Invalid")
        .when((col("dropoff_longitude") > -73.98) & (col("dropoff_latitude") > 40.75), "NE")
        .when((col("dropoff_longitude") > -73.98) & (col("dropoff_latitude") <= 40.75), "SE")
        .when((col("dropoff_longitude") <= -73.98) & (col("dropoff_latitude") > 40.75), "NW")
        .when((col("dropoff_longitude") <= -73.98) & (col("dropoff_latitude") <= 40.75), "SW")
        .otherwise("Unknown")
    )

# Añadir columnas de cuadrantes
df_with_quadrants = add_quadrants(df)

# Añadir columna del día de la semana
df_with_days = df_with_quadrants.withColumn(
    "day_of_week", 
    date_format(col("tpep_pickup_datetime"), "EEEE")
)

# Mostrar el esquema del dataframe modificado
print("\nEsquema del dataframe modificado:")
df_with_days.printSchema()

# Guardar el dataframe modificado para su uso posterior si es necesario
# df_with_days.write.mode("overwrite").parquet("path_to_save_modified_data")

# QUERY 1: Cuadrantes de origen y destino más comunes
print("\nQUERY 1: Cuadrantes de origen y destino más comunes")

# Los cuadrantes de origen más comunes
pickup_quadrants = df_with_days.groupBy("pickup_quadrant") \
    .count() \
    .orderBy(col("count").desc())

print("\nCuadrantes de origen más comunes:")
pickup_quadrants.show()

# Guardar resultados de cuadrantes de origen
pickup_quadrants.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/cuadrantes_origen")
print("Resultados guardados en el directorio 'resultados/cuadrantes_origen'")

# Los cuadrantes de destino más comunes
dropoff_quadrants = df_with_days.groupBy("dropoff_quadrant") \
    .count() \
    .orderBy(col("count").desc())

print("\nCuadrantes de destino más comunes:")
dropoff_quadrants.show()

# Guardar resultados de cuadrantes de destino
dropoff_quadrants.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/cuadrantes_destino")
print("Resultados guardados en el directorio 'resultados/cuadrantes_destino'")

# Las rutas más comunes (combinación de origen-destino)
routes = df_with_days.groupBy("pickup_quadrant", "dropoff_quadrant") \
    .count() \
    .orderBy(col("count").desc())

print("\nRutas más comunes (origen-destino):")
routes.show(10)

# Guardar resultados de rutas más comunes
routes.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/rutas_comunes")
print("Resultados guardados en el directorio 'resultados/rutas_comunes'")

# QUERY 2: Número de viajes e ingreso promedio por día de la semana
print("\nQUERY 2: Número de viajes e ingreso promedio por día de la semana")

trips_by_day = df_with_days.groupBy("day_of_week") \
    .agg(
        count("*").alias("trip_count"),
        avg("total_amount").alias("avg_income")
    ) \
    .orderBy(
        # Ordenar por día de la semana (lunes a domingo)
        when(col("day_of_week") == "Monday", 1)
        .when(col("day_of_week") == "Tuesday", 2)
        .when(col("day_of_week") == "Wednesday", 3)
        .when(col("day_of_week") == "Thursday", 4)
        .when(col("day_of_week") == "Friday", 5)
        .when(col("day_of_week") == "Saturday", 6)
        .when(col("day_of_week") == "Sunday", 7)
    )

print("\nNúmero de viajes e ingreso promedio por día de la semana:")
trips_by_day.show()

# Guardar los resultados como CSV para análisis posterior
trips_by_day.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/viajes_por_dia")
print("Resultados guardados en el directorio 'resultados/viajes_por_dia'")

# Análisis adicional: Promedio de propinas por día de la semana
tips_by_day = df_with_days.groupBy("day_of_week") \
    .agg(
        avg("tip_amount").alias("avg_tip"),
        avg(col("tip_amount") / col("fare_amount")).alias("tip_percentage")
    ) \
    .orderBy(
        when(col("day_of_week") == "Monday", 1)
        .when(col("day_of_week") == "Tuesday", 2)
        .when(col("day_of_week") == "Wednesday", 3)
        .when(col("day_of_week") == "Thursday", 4)
        .when(col("day_of_week") == "Friday", 5)
        .when(col("day_of_week") == "Saturday", 6)
        .when(col("day_of_week") == "Sunday", 7)
    )

print("\nPromedio de propinas por día de la semana:")
tips_by_day.show()

# Guardar los resultados de propinas como CSV
tips_by_day.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/propinas_por_dia")
print("Resultados guardados en el directorio 'resultados/propinas_por_dia'")

# Análisis adicional: Tiempo promedio de viaje por cuadrante
from pyspark.sql.functions import unix_timestamp, round as spark_round

print("\nAnálisis adicional: Tiempo promedio de viaje por cuadrante")
df_with_duration = df_with_days.withColumn(
    "trip_duration_minutes", 
    spark_round((unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60, 2)
)

# Tiempo promedio de viaje por cuadrante de origen
avg_duration_by_pickup = df_with_duration.groupBy("pickup_quadrant") \
    .agg(
        avg("trip_duration_minutes").alias("avg_duration_minutes"),
        count("*").alias("trip_count")
    ) \
    .orderBy(col("avg_duration_minutes").desc())

print("\nTiempo promedio de viaje por cuadrante de origen:")
avg_duration_by_pickup.show()

# Guardar resultados
avg_duration_by_pickup.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/duracion_por_origen")
print("Resultados guardados en el directorio 'resultados/duracion_por_origen'")

# Tiempo promedio de viaje por cuadrante de destino
avg_duration_by_dropoff = df_with_duration.groupBy("dropoff_quadrant") \
    .agg(
        avg("trip_duration_minutes").alias("avg_duration_minutes"),
        count("*").alias("trip_count")
    ) \
    .orderBy(col("avg_duration_minutes").desc())

print("\nTiempo promedio de viaje por cuadrante de destino:")
avg_duration_by_dropoff.show()

# Guardar resultados
avg_duration_by_dropoff.coalesce(1).write.mode("overwrite").option("header", "true").csv("resultados/duracion_por_destino")
print("Resultados guardados en el directorio 'resultados/duracion_por_destino'")

# Detener la sesión de Spark
spark.stop()

Muestra de los datos originales:
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875| 40.

+---------------+-------+
|pickup_quadrant|  count|
+---------------+-------+
|             NE|4606849|
|             SW|4264446|
|             NW|2292330|
|             SE| 865326|
|        Invalid| 182001|
+---------------+-------+



Resultados guardados en el directorio 'resultados/cuadrantes_origen'

Cuadrantes de destino más comunes:


+----------------+-------+
|dropoff_quadrant|  count|
+----------------+-------+
|              NE|4805320|
|              SW|3902685|
|              NW|2102949|
|              SE|1228838|
|         Invalid| 171160|
+----------------+-------+



Resultados guardados en el directorio 'resultados/cuadrantes_destino'

Rutas más comunes (origen-destino):


...

Resultados guardados en el directorio 'resultados/propinas_por_dia'

Análisis adicional: Tiempo promedio de viaje por cuadrante

Tiempo promedio de viaje por cuadrante de origen:


+---------------+--------------------+----------+
|pickup_quadrant|avg_duration_minutes|trip_count|
+---------------+--------------------+----------+
|             SE|  23.314068524463657|    865326|
|             SW|  15.351941243950597|   4264446|
|             NW|  15.202678981647523|   2292330|
|             NE|   15.08916382108472|   4606849|
|        Invalid|   7.816470568843067|    182001|
+---------------+--------------------+----------+



Resultados guardados en el directorio 'resultados/duracion_por_origen'

Tiempo promedio de viaje por cuadrante de destino:


+----------------+--------------------+----------+
|dropoff_quadrant|avg_duration_minutes|trip_count|
+----------------+--------------------+----------+
|              SE|  21.746755194744974|   1228838|
|              SW|   15.31795916144919|   3902685|
|              NW|  14.982833872813877|   2102949|
|              NE|  14.773323632973527|   4805320|
|         Invalid|  14.164315727973854|    171160|
+----------------+--------------------+----------+



Resultados guardados en el directorio 'resultados/duracion_por_destino'
